In [1]:
#tensorflow sentiment analysis

import pandas as pd
import numpy as np
import tensorflow as tf
import keras

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
import pickle
from collections import Counter

Using TensorFlow backend.


In [2]:
df=pd.read_csv('Hotel_Reviews.csv')

In [3]:
neg_rev=df[df['Negative_Review']!='No Negative'].Negative_Review.reset_index().drop('index',1)
pos_rev=df[df['Positive_Review']!='No Positive'].Positive_Review.reset_index().drop('index',1)

In [4]:
print(len(pos_rev),len(neg_rev))

479792 387848


In [5]:
neg_rev=list(neg_rev['Negative_Review'])
pos_rev=list(pos_rev['Positive_Review'])

In [6]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews 

#TokTok faster than word_tokenize
from nltk.tokenize import ToktokTokenizer
toktok = ToktokTokenizer()

# Stopwords, numbers and punctuation to remove
remove_punct_and_digits = dict([(ord(punct), ' ') for punct in string.punctuation + string.digits])
stopWords = set(stopwords.words('english'))

def word_cleaner(data):
    cleaned_word = data.lower().translate(remove_punct_and_digits)
    words = [toktok.tokenize(sent) for sent in sent_tokenize(cleaned_word)]
    wordsFiltered = []
    if not words:
        pass
    else:
        for w in words[0]:
            if w not in stopWords:
                wordsFiltered.append(w)

    return wordsFiltered

def rev_to_array(rev):
    features=np.zeros(feat_size)
    for w in word_cleaner(rev):
        if w in lexicon:
            index_val=lexicon.index(w)
            features[index_val]+=1
    return np.array([features])

In [7]:
#smaller set
neg_rev=neg_rev[:100000]
pos_rev=pos_rev[:100000]

# Create Lexicon to use
all_words=[]

for i in range(len(neg_rev)):
    all_words+=word_cleaner(neg_rev[i])
for i in range(len(pos_rev)):
    all_words+=word_cleaner(pos_rev[i])

count=Counter(all_words)
rel_words=[]

for w in count:
    if 100<count[w]<10000: 
        rel_words.append(w)

In [8]:
com_words=[]

for w in count:
    if 1000<count[w]: 
        com_words.append(w)
len(com_words)        

387

In [9]:
'better' in com_words

True

In [223]:
# SLOW! Create features vectors from Lexicon

feat_size=len(com_words)
lexicon=com_words
featureset=[]

for i in range(len(neg_rev)):
    features=np.zeros(feat_size)
    for w in word_tokenize(neg_rev[i]):
        if w in lexicon:
            index_val=lexicon.index(w)
            features[index_val]+=1
    featureset.append([features, [0,1]])
for i in range(len(pos_rev)):
    features=np.zeros(feat_size)
    for w in word_tokenize(pos_rev[i]):
        if w in lexicon:
            index_val=lexicon.index(w)
            features[index_val]+=1
    featureset.append([features, [1,0]])

In [224]:
# Shuffle and create CV and train sets
random.shuffle(featureset)

test_size=0.2
test_size=int(len(featureset)*test_size)

train_x=list(np.array(featureset)[:,0][:-test_size])
train_y=list(np.array(featureset)[:,1][:-test_size])

test_x=list(np.array(featureset)[:,0][-test_size:])
test_y=list(np.array(featureset)[:,1][-test_size:])

In [225]:
# Save sets to avoid re-running
with open('Sentiment_set.pickle','wb') as f:
    pickle.dump([train_x,train_y,test_x,test_y],f)

In [10]:
import pickle

[train_x,train_y,test_x,test_y] = pickle.load( open( "Sentiment_set.pickle", "rb" ) )

In [11]:
import tensorflow as tf
import keras
%matplotlib inline

#Model test number 1
model = keras.Sequential()
model.add(keras.layers.Dense(100, input_dim=len(train_x[0]),activation=tf.nn.relu))
#model.add(keras.layers.Dense(100,activation=tf.nn.relu))
model.add(keras.layers.Dense(2, activation=tf.nn.softmax))

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='categorical_crossentropy')

In [12]:
x_val = train_x[:2000]
partial_x_train = train_x[2000:1000000]
y_val = train_y[:2000]
partial_y_train = train_y[2000:1000000]

In [13]:
history = model.fit(np.array(partial_x_train),
                    np.array(partial_y_train),
                    epochs=5,
                    batch_size=300,
                    validation_data=(np.array(x_val), np.array(y_val)),
                    verbose=1)

Train on 158000 samples, validate on 2000 samples
Epoch 1/5
158000/158000 [==============================] - 15s 98us/step - loss: 0.3429 - val_loss: 0.3122
Epoch 2/5
158000/158000 [==============================] - 17s 108us/step - loss: 0.3030 - val_loss: 0.3029
Epoch 3/5
158000/158000 [==============================] - 14s 89us/step - loss: 0.2924 - val_loss: 0.2992
Epoch 4/5
158000/158000 [==============================] - 14s 89us/step - loss: 0.2843 - val_loss: 0.2983
Epoch 5/5
158000/158000 [==============================] - 17s 105us/step - loss: 0.2770 - val_loss: 0.2967


In [ ]:
from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

In [288]:
# Test with a sentence of your choise
my_rev=rev_to_array('Just a bit noisy')
print(model.predict(my_rev))
my_rev=rev_to_array('great place to stay')
print(model.predict(my_rev))
my_rev=rev_to_array('I am sure to go back again')
print(model.predict(my_rev))

[[ 0.00245318  0.99754685]]
[[ 0.75243831  0.24756172]]
[[ 0.37685296  0.62314707]]


In [279]:
scores=[]
for w in lexicon:
    my_rev=rev_to_array(w)
    scores.append(model.predict(my_rev)[0][0])

In [280]:
words_scores=pd.DataFrame([lexicon,scores]).T

In [284]:
display(words_scores.sort_values(1,ascending=False).head(10))
display(words_scores.sort_values(1,ascending=False).tail(10))

,0,1
373,comfy,0.970558
258,location,0.965154
109,birthday,0.950797
303,spacious,0.945313
304,quiet,0.943517
286,beautiful,0.941095
350,welcome,0.939178
284,helpful,0.918985
177,amazing,0.902039
345,attentive,0.901904


,0,1
12,small,0.0214666
87,noisy,0.0211334
202,cold,0.0197974
295,charged,0.0191183
213,dark,0.0185563
282,slow,0.0114165
61,difficult,0.0106576
199,uncomfortable,0.0100351
319,tiny,0.00967075
60,bit,0.00943842
